#### **Output Parser**

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_community.llms import Ollama

In [6]:
chat = Ollama(model='llama3')

/tmp/ipykernel_167511/1136294162.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  chat = Ollama(model='llama3')


In [7]:
response_schemas = [
    ResponseSchema(name='bad_string', description='This is a poorly formatted user input string'),
    ResponseSchema(name='good_string', description='This is your response, a reformatted response'),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [8]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [9]:
template = """
You are will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

%USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=['user_input'],
    partial_variables={"format_instructions" : format_instructions},
    template=template
)

promptValue = prompt.format(user_input="Welcome to Californiya!")
print(promptValue)


You are will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

%USER INPUT:
Welcome to Californiya!

YOUR RESPONSE:



In [10]:
llm_output = chat.invoke(promptValue)

In [11]:
print(llm_output)

Here's the reformatted output:

```json
{
    "bad_string": "Welcome to Californiya!",
    "good_string": "Welcome to California!"  // Reformatted string with correct spelling of state name
}
```


In [13]:
parsed_output = output_parser.parse(llm_output)
print(parsed_output)

OutputParserException: Got invalid JSON object. Error: Expecting ',' delimiter: line 3 column 46 (char 92)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

#### **Text Splitters**
Often times your document is too long (like a book) for your LLM. You need to split it up into chunks. Text splitters help with this.

There are many ways you could split your text into chunks, experiment with [different ones](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) to see which is best for you.

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('hp.txt', 'r') as f:
    data = f.read()

print(f'Length of the data : {len([data])}')

Length of the data : 1


In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
) 

texts = text_splitter.create_documents([data]) 

In [16]:
print(f'You have {len(texts)} documents.')

You have 532 documents.


#### **Retrievers**
Easy way to combine documents with language models.

There are many different types of retrievers, the most widely supported is the VectoreStoreRetriever

In [17]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.embeddings import OllamaEmbeddings # Depricated 
from langchain_ollama.embeddings import OllamaEmbeddings

loader = TextLoader('hp.txt')
documents = loader.load()

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

texts = text_splitter.split_documents(documents)

embeddings = OllamaEmbeddings(model='llama3')

db = FAISS.from_documents(texts[:1], embeddings)

In [19]:
retriever = db.as_retriever()

In [20]:
docs = retriever.get_relevant_documents('What is the place name here')

/tmp/ipykernel_167511/2008780841.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents('What is the place name here')


In [21]:
docs, len(docs)

([Document(id='6a72a1c6-1e21-42ed-b965-86fe19c3a9f0', metadata={'source': 'hp.txt'}, page_content="Harry Potter and the Sorcerer's Stone\n\n\nCHAPTER ONE\n\nTHE BOY WHO LIVED\n\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you'd expect to be involved in anything strange or mysterious,\nbecause they just didn't hold with such nonsense.\n\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. The Dursleys had a small son called Dudley and in their\nopinion there was no finer boy anywhere.")],
 1)

In [22]:
l = "\n".join([x.page_content for x in docs])
print(l)

Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.

Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.


#### **Chat Message History**

In [30]:
from langchain.memory import ChatMessageHistory
from langchain_community.llms import Ollama

chat = Ollama(model='llama3')

history = ChatMessageHistory()

history.add_ai_message('Hi')

history.add_user_message('What is a capital for India?')

In [31]:
history.messages

[AIMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is a capital for India?', additional_kwargs={}, response_metadata={})]

In [32]:
ai_response = chat.invoke(history.messages)
ai_response

'The capital of India is New Delhi.'

In [33]:
history.add_ai_message(ai_response)

In [34]:
history.messages

[AIMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is a capital for India?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={})]

In [35]:
while True:
    history.add_user_message(input('User Prompt: '))
    ai_response = chat.invoke(history.messages)
    history.add_ai_message(ai)
    print(f"AI Response: {ai_response}")

User Prompt:  Can you tell me What to do in New Delhi in one day, Any fun activities. pls provide in bullets


AI Response: Here are some fun activities and attractions you can explore in New Delhi in one day:

* **Red Fort (Lal Qila)**: Start your day by exploring the magnificent Red Fort, a UNESCO World Heritage Site and a symbol of Indian history.
* **Qutub Minar**: Visit the tallest minaret in India, Qutub Minar, which offers stunning views of the city.
* **India Gate**: Take a stroll around the historic India Gate war memorial, built to commemorate the soldiers who lost their lives during World War I.
* **President's Estate (Rashtrapati Bhavan)**: Take a guided tour of the President's official residence and office building, which is an architectural marvel.
* **National Museum**: Visit the National Museum to learn about Indian history, art, and culture through its extensive collection of artifacts and exhibits.
* **Connaught Place**: End your day by exploring the bustling Connaught Place, a popular shopping and dining destination in the heart of Delhi.

Some other tips and suggestions:

* 

User Prompt:  any chat shops if you know


AI Response: Here are some popular chat shops and cafes in Connaught Place, New Delhi:

* **Café Coffee Day**: A popular coffee chain with multiple outlets in CP.
* **The Coffee Bean & Tea Leaf**: Another well-known international coffee brand with a presence in CP.
* **Barista**: An Indian coffee shop chain with a wide range of coffee and snack options.
* **Costa Coffee**: A UK-based coffee chain with a strong presence in India, including multiple outlets in CP.
* **Theobroma**: A popular dessert cafe with unique and delicious treats like chocolate chip cookies and brownies.
* **Blue Tokai**: A specialty coffee shop with high-quality Arabica beans and expertly brewed cups.

These are just a few examples of the many chat shops and cafes you can find in Connaught Place. You can also explore local favorites, try street food, or indulge in traditional Indian snacks like chaat, kebabs, or samosas!

Remember to stay hydrated and energized throughout your day of exploring New Delhi!


User Prompt:  No, I am talking about the chaat shops or english we say fast fodd


AI Response: I apologize for the confusion!

In that case, here are some popular chaat shops (also known as street food stalls) in Connaught Place, New Delhi:

* **Aalaya Chaat**: A well-known chaat stall serving delicious snacks like crispy samosas, crunchy kachoris, and flavorful chole bhatures.
* **Sister's Fast Food**: A popular fast-food joint offering a wide range of street food, including chaats, kebabs, sandwiches, and more.
* **Al Jawahar**: A famous Delhi-style chaat stall serving mouthwatering snacks like pani puri, samosas, and kachoris.
* **Khoobsurat Chaat Bhandar**: Another popular chaat shop with a variety of options, including innovative creations like the "Dahi Vada" (a yogurt-based snack).
* **Saffron Fast Food**: A quick-service restaurant serving a range of fast food, including chaats, kebabs, and sandwiches.

These are just a few examples of the many chaat shops you can find in Connaught Place. Be prepared for a flavorful and filling experience!

Remember to wash 

User Prompt:  Thanks


AI Response: You're welcome! It was my pleasure to help you with information about New Delhi and its popular attractions, chat shops (fast food), and cafes. I hope you have a fantastic time exploring the city! If you have any more questions or need further assistance, feel free to ask!


KeyboardInterrupt: Interrupted by user